## 뉴스 Crawling ㄱㄱ

In [1]:
# 일단 다 긁어온거라 추후에 필요없는건 삭제 필요요

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import math


# --------------------------------------
# 웹페이지 열고 이동하기 
# --------------------------------------
driver = Chrome()

# 브라우저 크기 최대화
driver.maximize_window()

In [5]:
# --------------------------------------
# 종목 코드 이용하여 주식 뉴스 페이지 들어가기
# --------------------------------------
stockCode = '005930' 	# 삼성전자 종목 코드
startPage = 1			# 뉴스 수집 시작 페이지
lastPage = 200 			# 뉴스 수집 마지막 페이지

In [6]:
# news_data 초기화
news_data = []

# 시작 페이지부터 마지막 페이지까지 순서대로 뉴스 데이터 수집집
for pageNum in range(startPage, lastPage + 1):
	url = f'https://finance.naver.com/item/news.naver?code={stockCode}&page={pageNum}'
	driver.get(url)

	# iframe이 로드될 때까지 대기
	try: 
		WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "news_frame")))

		# iframe으로 전환 (name 또는 id 사용 가능)
		driver.switch_to.frame("news_frame") # 여기선 id 사용

		# 모든 tr 선택
		rows = driver.find_elements(By.CSS_SELECTOR, "table.type5 > tbody > tr")

		for i, row in enumerate(rows):
			# tr의 class 속성 가져오기
			tr_class = row.get_attribute("class") or ""
			if "relation_lst" in tr_class:
				continue  # relation_lst가 포함된 tr은 건너뜀  ---> 제대로 작동 안 됨
			
			# 메인 기사만 추출
			tds = row.find_elements(By.TAG_NAME, "td")
			if len(tds) >= 3:
				try:
					a_tag = tds[0].find_element(By.TAG_NAME, "a")
					newsTitle = a_tag.text
					newsAddress = a_tag.get_attribute("href")
				except:
					newsTitle = ""
					newsAddress = ""
				
				if newsTitle.strip():
					newsFrom = tds[1].text
					newsWhen = tds[2].text.strip()

					if " " in newsWhen:
						newsDate, newsTime = newsWhen.split(" ", 1)
					else:
						newsDate = newsWhen
						newsTime = ""
						
					#print(newsTitle, newsFrom, newsDate)
					news_data.append({'date': newsDate, 'time' : newsTime, 'title': newsTitle, 'from': newsFrom, 'link' : newsAddress})


		print(f"페이지 {pageNum} 완료 - 현재까지 {len(news_data)}개 뉴스")

		
	except Exception as e:
		print(f"페이지 {pageNum}에서 오류 발생: {e}")

	finally:
		# 다음 페이지를 위해 메인 프레임으로 복귀 
		driver.switch_to.default_content()

df = pd.DataFrame(news_data)
df

페이지 1 완료 - 현재까지 13개 뉴스
페이지 2 완료 - 현재까지 24개 뉴스
페이지 3 완료 - 현재까지 34개 뉴스
페이지 4 완료 - 현재까지 48개 뉴스
페이지 5 완료 - 현재까지 61개 뉴스
페이지 6 완료 - 현재까지 72개 뉴스
페이지 7 완료 - 현재까지 84개 뉴스
페이지 8 완료 - 현재까지 95개 뉴스
페이지 9 완료 - 현재까지 105개 뉴스
페이지 10 완료 - 현재까지 115개 뉴스
페이지 11 완료 - 현재까지 128개 뉴스
페이지 12 완료 - 현재까지 141개 뉴스
페이지 13 완료 - 현재까지 157개 뉴스
페이지 14 완료 - 현재까지 171개 뉴스
페이지 15 완료 - 현재까지 185개 뉴스
페이지 16 완료 - 현재까지 199개 뉴스
페이지 17 완료 - 현재까지 213개 뉴스
페이지 18 완료 - 현재까지 224개 뉴스
페이지 19 완료 - 현재까지 234개 뉴스
페이지 20 완료 - 현재까지 245개 뉴스
페이지 21 완료 - 현재까지 256개 뉴스
페이지 22 완료 - 현재까지 268개 뉴스
페이지 23 완료 - 현재까지 278개 뉴스
페이지 24 완료 - 현재까지 295개 뉴스
페이지 25 완료 - 현재까지 308개 뉴스
페이지 26 완료 - 현재까지 321개 뉴스
페이지 27 완료 - 현재까지 335개 뉴스
페이지 28 완료 - 현재까지 349개 뉴스
페이지 29 완료 - 현재까지 364개 뉴스
페이지 30 완료 - 현재까지 379개 뉴스
페이지 31 완료 - 현재까지 395개 뉴스
페이지 32 완료 - 현재까지 411개 뉴스
페이지 33 완료 - 현재까지 425개 뉴스
페이지 34 완료 - 현재까지 439개 뉴스
페이지 35 완료 - 현재까지 452개 뉴스
페이지 36 완료 - 현재까지 464개 뉴스
페이지 37 완료 - 현재까지 474개 뉴스
페이지 38 완료 - 현재까지 485개 뉴스
페이지 39 완료 - 현재까지 496개 뉴스
페이지 40 완료 - 현재까지 508개 뉴스
페이지 41 완료 - 현재까지 

,date,time,title,from,link
0,2025.06.02,09:30,"삼성페이 오전부터 일부 결제 오류...""원인 파악 중""",아이뉴스24,https://n.news.naver.com/mnews/article/031/000...
1,2025.06.02,09:28,"""월요일 아침부터"" 삼성페이 결제 오류 발생… ""정확한 원인 파악 중...",머니S,https://n.news.naver.com/mnews/article/417/000...
2,2025.06.02,09:30,"대선 하루 앞둔 코스피 장중 2,710대서 강세…철강株 하락(종합)",연합뉴스,https://n.news.naver.com/mnews/article/001/001...
3,2025.06.02,09:25,“출근길 편의점 갔다 당황”…삼성페이 1시간 넘게 결제 장애,세계일보,https://n.news.naver.com/mnews/article/022/000...
4,2025.06.02,08:37,삼성페이 결제 장애 발생,연합뉴스,https://n.news.naver.com/mnews/article/001/001...
...,...,...,...,...,...
2682,2025.05.23,15:34,"코스피, 외인·기관 쌍끌이 매도에 하락 마감 [fn마감시황]",파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...
2683,2025.05.23,15:28,'삼성 신약' 주도하는 기술통… 김경아 에피스 사장,머니S,https://n.news.naver.com/mnews/article/417/000...
2684,2025.05.23,15:23,"'시밀러 13년' 삼성바이오에피스, K신약 개발 '속도'",머니S,https://n.news.naver.com/mnews/article/417/000...
2685,2025.05.23,15:20,"예보, MG손보 정리 착수…가교보험사 자본금 300억 출자",머니투데이,https://n.news.naver.com/mnews/article/008/000...


In [17]:
# DataFrame을 csv 파일로 출력력

uniqueTime = datetime.now().strftime('%Y%m%d_%H%M%S')
file_path_csv = f'data/{uniqueTime}_news.csv'
#file_path_excel = f'data/{uniqueTime}_news.xlsx'

df.to_csv(file_path_csv, index = False, encoding = "utf-8-sig")
print(f"{startPage}페이지부터 {lastPage}페이지까지 총 {lastPage - startPage + 1}페이지에서 {(len(df))}개의의 뉴스 기사를 CSV 파일({file_path_csv})로 저장하였습니다.")

1페이지부터 50페이지까지 총 50페이지에서 635개의의 뉴스 기사를 CSV 파일(data/20250530_163841_news.csv)로 저장하였습니다.


In [ ]:
### 여기서부턴 초기버전 코드. 필요 X 

In [ ]:
url = f'https://finance.naver.com/item/news.naver?code={stockCode}&page={pageNum}'
driver.get(url)

In [ ]:
# iframe이 로드될 때까지 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "news_frame"))
)

In [ ]:
# iframe으로 전환 (name 또는 id 사용 가능)
driver.switch_to.frame("news_frame")  


In [ ]:
# 모든 tr 선택
rows = driver.find_elements(By.CSS_SELECTOR, "table.type5 > tbody > tr")

for i, row in enumerate(rows):
    # tr의 class 속성 가져오기
    tr_class = row.get_attribute("class")
    if tr_class and "relation_lst_cluster" in tr_class:
        continue  # relation_lst가 포함된 tr은 건너뜀  ---> 제대로 작동 안 됨

    tds = row.find_elements(By.TAG_NAME, "td")
    if len(tds) >= 3:
        try:
            newsTitle = tds[0].find_element(By.TAG_NAME, "a").text
        except:
            newsTitle = ""
        newsFrom = tds[1].text
        newsDate = tds[2].text
        print(newsTitle, newsFrom, newsDate)
        news_data.append({'title': newsTitle, 'from': newsFrom, 'date': newsDate})

news_data

In [ ]:
df = pd.DataFrame(news_data)
df